In [1]:
!git clone https://github.com/dml-qom/FarsTail.git


Cloning into 'FarsTail'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 125 (delta 19), reused 18 (delta 18), pack-reused 104
Receiving objects: 100% (125/125), 43.86 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [2]:
import pandas as pd
import numpy as np

In [7]:
train_data = pd.read_csv('/content/FarsTail/data/Train-word.csv', sep='\t')
val_data = pd.read_csv('/content/FarsTail/data/Val-word.csv', sep='\t')
test_data = pd.read_csv('/content/FarsTail/data/Test-word.csv', sep='\t')

In [3]:
!pip install transformers
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.7 MB/s eta 0:00:00


Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
label_encoder = {'e':0,'c':1,'n':2}
y_train = [label_encoder[i] for i in train_data['label'].values.tolist()]
y_valid = [label_encoder[i] for i in val_data['label'].values.tolist()]
y_test = [label_encoder[i] for i in test_data['label'].values.tolist()]

In [9]:
def Tokenizer_persian(data):
  return tokenizer(data, padding=True, truncation=True, max_length=32)

train_p = Tokenizer_persian(train_data['premise'].values.tolist())
train_h = Tokenizer_persian(train_data['hypothesis'].values.tolist())

valid_p = Tokenizer_persian(val_data['premise'].values.tolist())
valid_h = Tokenizer_persian(val_data['hypothesis'].values.tolist())

test_p = Tokenizer_persian(test_data['premise'].values.tolist())
test_h = Tokenizer_persian(test_data['hypothesis'].values.tolist())


In [10]:
train = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}
valid = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}
test = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}

for i in range(len(train_p['input_ids'])):
  train['input_ids'].append(train_p['input_ids'][i]+train_h['input_ids'][i])
  train['token_type_ids'].append(train_p['token_type_ids'][i]+train_h['token_type_ids'][i])
  train['attention_mask'].append(train_p['attention_mask'][i]+train_h['attention_mask'][i])

for i in range(len(valid_p['input_ids'])):
  valid['input_ids'].append(valid_h['input_ids'][i]+valid_p['input_ids'][i])
  valid['token_type_ids'].append(valid_h['token_type_ids'][i]+valid_p['token_type_ids'][i])
  valid['attention_mask'].append(valid_h['attention_mask'][i]+valid_p['attention_mask'][i])


for i in range(len(test_h['input_ids'])):
  test['input_ids'].append(test_p['input_ids'][i]+test_h['input_ids'][i])
  test['token_type_ids'].append(test_p['token_type_ids'][i]+test_h['token_type_ids'][i])
  test['attention_mask'].append(test_p['attention_mask'][i]+test_h['attention_mask'][i])


In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
model_name = "HooshvareLab/bert-fa-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

In [ ]:
train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(train, y_train)
valid_dataset = Dataset(valid, y_valid)
test_dataset = Dataset(test)

In [13]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [14]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  636628 KB |  636628 KB |  636628 KB |       0 B  |\n|       from large pool |  636128 KB |  636128 KB |  636128 KB |       0 B  |\n|       from small pool |     500 KB |     500 KB |     500 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  636628 KB |  636628 KB |  636628 KB |       0 B  |\n|       from large pool |  636128 KB |  636128 KB |

In [15]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7266
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2275
  Number of trainable parameters = 162843651


Step,Training Loss,Validation Loss
500,0.851100,2.092767
1000,0.507200,2.472534
1500,0.250600,3.541868
2000,0.125600,3.887858


***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin


Training completed. Do not fo

TrainOutput(global_step=2000, training_loss=0.4336355075836182, metrics={'train_runtime': 526.9002, 'train_samples_per_second': 68.95, 'train_steps_per_second': 4.318, 'total_flos': 1050611876987904.0, 'train_loss': 0.4336355075836182, 'epoch': 4.4})

In [23]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
trained_bert = BertForMaskedLM.from_pretrained('HooshvareLab/bert-fa-base-uncased')
tokenizer_trained_bert = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model.to('cuda')
trained_bert.to('cuda')
trained_bert.bert.encoder.layer=model.bert.encoder.layer

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcdde570ae11986a534542c2995a62/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcdde570ae11986a534542c2995a62/pytorch_model.bin
Generate config GenerationConfi

In [27]:
def predict_masked_word(sentence, modek,relationship=0):
    """ Predicts the masked word in a given sentence using a transformer model and it's tokenizer. """

    # Tokenize the input sentence  
    tokenized_sentence = tokenizer.tokenize("[CLS] " + sentence + "[SEP]")
    mask_index = tokenized_sentence.index("[MASK]")
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    token_tensor = torch.tensor([token_ids]).to('cuda')

    # Forward pass without training and backpropagation
    with torch.no_grad():
        outputs = modek(token_tensor)
        prediction_logits = outputs[0]
    
    # Calculating 3 nearest prediction of masked token 
    prediction_probs = torch.nn.functional.softmax(prediction_logits[0, mask_index], dim=-1)
    top_k_probs, top_k_token_ids = torch.topk(prediction_probs, 3, sorted=True)

    # Print the predicted tokens for mask and their weight
    predicted_mask = []
    for i in range(3):
        predicted_mask.append(tokenizer.convert_ids_to_tokens([top_k_token_ids[i]])[0])
        token_prob = top_k_probs[i].item()
        print(f"[MASK]: '{predicted_mask[i]}'  : {token_prob}")       
    return predicted_mask[0]   

In [21]:
import pandas as pd
df = pd.read_csv("/content/Test_file.txt", engine='python',encoding='utf-8', error_bad_lines=False)
df.columns = ["cat", "first", "second", "third","fourth"]
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,cat,first,second,third,fourth
0,city,اهواز,خوزستان,قشم,هرمزگان
1,city,شیراز,فارس,دامغان,سمنان
2,city,رشت,گیلان,اسلام‌شهر,تهران
3,city,بندرعباس,هرمزگان,ملارد,تهران
4,city,اراک,مرکزی,مریوان,کردستان


In [28]:
Accuracy_city = 0
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "شهر "+df["first"][i[0]] + " در استان " + df["second"][i[0]] + " است در نتیجه شهر " + df["third"][i[0]] + " در استان " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_city = Accuracy_city + 1   
print(Accuracy_city)        



اهواز خوزستان قشم هرمزگان
شهر اهواز در استان خوزستان است در نتیجه شهر قشم در استان [MASK] است.
[MASK]: 'خوزستان'  : 0.21807250380516052
[MASK]: 'یمن'  : 0.09598516672849655
[MASK]: 'هرمزگان'  : 0.03394705802202225
شیراز فارس دامغان سمنان
شهر شیراز در استان فارس است در نتیجه شهر دامغان در استان [MASK] است.
[MASK]: 'فارس'  : 0.06923981010913849
[MASK]: 'صلاح'  : 0.044315971434116364
[MASK]: 'واقع'  : 0.03236749768257141
رشت گیلان اسلام‌شهر تهران
شهر رشت در استان گیلان است در نتیجه شهر اسلام‌شهر در استان [MASK] است.
[MASK]: 'واقع'  : 0.03597154840826988
[MASK]: 'شناسایی'  : 0.027847429737448692
[MASK]: 'یمن'  : 0.017839724197983742
بندرعباس هرمزگان ملارد تهران
شهر بندرعباس در استان هرمزگان است در نتیجه شهر ملارد در استان [MASK] است.
[MASK]: 'هرمزگان'  : 0.9518794417381287
[MASK]: 'بندرعباس'  : 0.004420446697622538
[MASK]: 'یمن'  : 0.002228358294814825
اراک مرکزی مریوان کردستان
شهر اراک در استان مرکزی است در نتیجه شهر مریوان در استان [MASK] است.
[MASK]: 'مرکزی'  : 0.40856701135635376
[MASK

In [30]:
Accuracy_currency = 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " واحد پول "+df["second"][i[0]] + " برای کشور " + df["first"][i[0]] + " است پس ارز و پول " + "[MASK]" + " برای کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_currency = Accuracy_currency + 1   
num = y["cat"].count()      
print(Accuracy_currency/num)     

سوئد کرون امارات درهم
 واحد پول کرون برای کشور سوئد است پس ارز و پول [MASK] برای کشور امارات است.
[MASK]: 'ان'  : 0.2681635618209839
[MASK]: 'هم'  : 0.07729480415582657
[MASK]: 'نیز'  : 0.06818802654743195
سوئیس فرانک تاجیکستان سامانی
 واحد پول فرانک برای کشور سوئیس است پس ارز و پول [MASK] برای کشور تاجیکستان است.
[MASK]: 'ان'  : 0.13796521723270416
[MASK]: 'نیز'  : 0.0905325785279274
[MASK]: 'رسمی'  : 0.06509127467870712
رومانی لئو بریتانیا پوند
 واحد پول لئو برای کشور رومانی است پس ارز و پول [MASK] برای کشور بریتانیا است.
[MASK]: 'ان'  : 0.44058653712272644
[MASK]: 'ترکیه'  : 0.12699896097183228
[MASK]: 'نیز'  : 0.02900032140314579
ترکیه لیره افغانستان افغانی
 واحد پول لیره برای کشور ترکیه است پس ارز و پول [MASK] برای کشور افغانستان است.
[MASK]: 'نیز'  : 0.16003136336803436
[MASK]: 'ان'  : 0.1508907824754715
[MASK]: 'ترکیه'  : 0.11647321283817291
چین یوان عراق دینار
 واحد پول یوان برای کشور چین است پس ارز و پول [MASK] برای کشور عراق است.
[MASK]: 'ان'  : 0.3069421052932739
[MASK]: 'ما

In [31]:
Accuracy_capital = 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " شهر "+df["second"][i[0]] + " پایتخت کشور " + df["first"][i[0]] + " است پس شهر " + "[MASK]" + " پایتخت کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_capital = Accuracy_capital + 1   
num = y["cat"].count()      
print(Accuracy_capital/num)     

روسیه مسکو هلند آمستردام
 شهر مسکو پایتخت کشور روسیه است پس شهر [MASK] پایتخت کشور هلند است.
[MASK]: 'امستردام'  : 0.10633896291255951
[MASK]: 'برلین'  : 0.0920039638876915
[MASK]: 'گرمی'  : 0.034864626824855804
کره‌جنوبی سئول آمریکا واشنگتن
 شهر سئول پایتخت کشور کره‌جنوبی است پس شهر [MASK] پایتخت کشور آمریکا است.
[MASK]: 'سيول'  : 0.7037056684494019
[MASK]: 'بوسان'  : 0.04046507924795151
[MASK]: 'کرهای'  : 0.02279767207801342
ایران تهران اسپانیا مادرید
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور اسپانیا است.
[MASK]: 'تهران'  : 0.9232966303825378
[MASK]: 'ایران'  : 0.008435851894319057
[MASK]: 'قم'  : 0.004381333477795124
ایران تهران افغانستان کابل
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور افغانستان است.
[MASK]: 'تهران'  : 0.9400628805160522
[MASK]: 'قم'  : 0.005104892421513796
[MASK]: 'ایران'  : 0.0035795189905911684
بریتانیا لندن فنلاند هلسینکی
 شهر لندن پایتخت کشور بریتانیا است پس شهر [MASK] پایتخت کشور فنلاند است.
[MASK]: 'لندن'  : 0.1251917779445648

In [32]:
Accuracy_family = 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت خانوادگی "+df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت خانوادگی " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_family = Accuracy_family + 1   
num = y["cat"].count()      
print(Accuracy_family/num)     

برادر خواهر پدر مادر
 نسبت خانوادگی برادر به خواهر مانند نسبت خانوادگی پدر به [MASK] است.
[MASK]: 'خواهر'  : 0.2758188843727112
[MASK]: 'برادر'  : 0.14455701410770416
[MASK]: 'خواهران'  : 0.022307299077510834
برادرها خواهرها شوهر عیال
 نسبت خانوادگی برادرها به خواهرها مانند نسبت خانوادگی شوهر به [MASK] است.
[MASK]: 'این'  : 0.0286794975399971
[MASK]: 'خواهر'  : 0.025548342615365982
[MASK]: 'برادر'  : 0.02501937933266163
پدربزرگ مادربزرگ برادر خواهر
 نسبت خانوادگی پدربزرگ به مادربزرگ مانند نسبت خانوادگی برادر به [MASK] است.
[MASK]: 'مادربزرگ'  : 0.03436431661248207
[MASK]: 'خواهر'  : 0.03204882889986038
[MASK]: 'پدربزرگ'  : 0.03194936364889145
بابابزرگ مامان‌بزرگ داماد عروس
 نسبت خانوادگی بابابزرگ به مامان‌بزرگ مانند نسبت خانوادگی داماد به [MASK] است.
[MASK]: 'پدر'  : 0.029312381520867348
[MASK]: 'خوبی'  : 0.028376374393701553
[MASK]: 'زنان'  : 0.023790298029780388
داماد عروس برادرها خواهرها
 نسبت خانوادگی داماد به عروس مانند نسبت خانوادگی برادرها به [MASK] است.
[MASK]: 'داخل'  : 0.0809

In [33]:
Accuracy = 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " فعل "+df["first"][i[0]] + " سوم شخص مفرد و فعل " + df["second"][i[0]] + " سوم شخص جمع است در نتیجه فعل " + df["third"][i[0]] + " سوم شخص مفرد و فعل " + "[MASK]" + " سوم شخص جمع است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)     

برید بریدند رقصید رقصیدند
 فعل برید سوم شخص مفرد و فعل بریدند سوم شخص جمع است در نتیجه فعل رقصید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'سازی'  : 0.039872292429208755
[MASK]: 'من'  : 0.027759501710534096
[MASK]: 'کردم'  : 0.017282478511333466
پرید پریدند آمد آمدند
 فعل پرید سوم شخص مفرد و فعل پریدند سوم شخص جمع است در نتیجه فعل آمد سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'سازی'  : 0.03760701045393944
[MASK]: 'شناسایی'  : 0.019653035327792168
[MASK]: 'ما'  : 0.01919582299888134
تازید تازیدند گفت گفتند
 فعل تازید سوم شخص مفرد و فعل تازیدند سوم شخص جمع است در نتیجه فعل گفت سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'من'  : 0.052795544266700745
[MASK]: 'ما'  : 0.030947593972086906
[MASK]: 'سازی'  : 0.02787461131811142
شد شدند بوسید بوسیدند
 فعل شد سوم شخص مفرد و فعل شدند سوم شخص جمع است در نتیجه فعل بوسید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'سازی'  : 0.09038349986076355
[MASK]: 'کردن'  : 0.028132621198892593
[MASK]: 'شناسایی'  : 0.0264641921967

In [34]:
Accuracy = 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  مصدر "+df["third"][i[0]] + " فعل " + "[MASK]" + " است همانطور که مصدر " + df["first"][i[0]] + " فعل " + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)      

بریدن برید دمیدن دمید
  مصدر دمیدن فعل [MASK] است همانطور که مصدر بریدن فعل برید است.
[MASK]: 'ازاد'  : 0.06135467067360878
[MASK]: 'گاز'  : 0.018605316057801247
[MASK]: 'باز'  : 0.012203962542116642
تازیدن تاخت دادن داد
  مصدر دادن فعل [MASK] است همانطور که مصدر تازیدن فعل تاخت است.
[MASK]: 'ازاد'  : 0.02392481081187725
[MASK]: 'اتش'  : 0.023644905537366867
[MASK]: 'ترک'  : 0.01786975935101509
شدن شد نگریستن نگریست
  مصدر نگریستن فعل [MASK] است همانطور که مصدر شدن فعل شد است.
[MASK]: 'شده'  : 0.11021154373884201
[MASK]: 'شدن'  : 0.045564137399196625
[MASK]: 'کردن'  : 0.037337154150009155
گفتن گفت رقصیدن رقصید
  مصدر رقصیدن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: 'خوبی'  : 0.07583977282047272
[MASK]: 'گفتن'  : 0.05878627300262451
[MASK]: 'گفت'  : 0.040221329778432846
گفتن گفت خواستن خواست
  مصدر خواستن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: 'خوبی'  : 0.10569324344396591
[MASK]: 'گفتن'  : 0.08637119829654694
[MASK]: 'خوب'  : 0.033032916486263275
توانست

In [35]:
Accuracy = 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " من "+df["third"][i[0]] + "  پس ما " + "[MASK]" + " من " + df["first"][i[0]] + " و ما " + df["second"][i[0]] + " "
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

دادم دادیم یافتم یافتیم
 من یافتم  پس ما [MASK] من دادم و ما دادیم 
[MASK]: 'را'  : 0.4911980628967285
[MASK]: 'به'  : 0.13719141483306885
[MASK]: 'گفتم'  : 0.04083312302827835
زدم زدیم نامیدم نامیدیم
 من نامیدم  پس ما [MASK] من زدم و ما زدیم 
[MASK]: 'را'  : 0.45348912477493286
[MASK]: 'هم'  : 0.051835887134075165
[MASK]: 'و'  : 0.029712192714214325
نوشتم نوشتیم گفتم گفتیم
 من گفتم  پس ما [MASK] من نوشتم و ما نوشتیم 
[MASK]: 'را'  : 0.21119894087314606
[MASK]: '،'  : 0.11300299316644669
[MASK]: 'هم'  : 0.07313597947359085
دانستم دانستیم بلعیدم بلعیدیم
 من بلعیدم  پس ما [MASK] من دانستم و ما دانستیم 
[MASK]: 'را'  : 0.7520444393157959
[MASK]: '،'  : 0.0136336050927639
[MASK]: 'هم'  : 0.011723075062036514
گردیدم گردیدیم یافتم یافتیم
 من یافتم  پس ما [MASK] من گردیدم و ما گردیدیم 
[MASK]: 'و'  : 0.08187514543533325
[MASK]: '،'  : 0.08179624378681183
[MASK]: '.'  : 0.06673837453126907
گردیدم گردیدیم بوسیدم بوسیدیم
 من بوسیدم  پس ما [MASK] من گردیدم و ما گردیدیم 
[MASK]: 'و'  : 0.130581319

In [36]:
Accuracy = 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  "+ df["first"][i[0]] + " صفت و " + df["second"][i[0]] + " قید است. در نتیجه " + df["third"][i[0]] + " صفت و " + "[MASK]" + " قید است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

مجدد مجددا مخصوص مخصوصا
  مجدد صفت و مجددا قید است. در نتیجه مخصوص صفت و [MASK] قید است.
[MASK]: 'نیز'  : 0.154011532664299
[MASK]: 'مجددا'  : 0.13947361707687378
[MASK]: 'هم'  : 0.0359518900513649
عمده عمدتا صمیمی صمیمانه
  عمده صفت و عمدتا قید است. در نتیجه صمیمی صفت و [MASK] قید است.
[MASK]: 'عمدتا'  : 0.3168202340602875
[MASK]: 'اکثرا'  : 0.07731263339519501
[MASK]: 'غالبا'  : 0.07472342252731323
دقیق دقیقا عادل عادلانه
  دقیق صفت و دقیقا قید است. در نتیجه عادل صفت و [MASK] قید است.
[MASK]: 'دقیقا'  : 0.09640657156705856
[MASK]: 'هم'  : 0.08832989633083344
[MASK]: 'واقعا'  : 0.07148051261901855
غالب غالبا جدا جداگانه
  غالب صفت و غالبا قید است. در نتیجه جدا صفت و [MASK] قید است.
[MASK]: 'غالبا'  : 0.8168620467185974
[MASK]: 'معمولا'  : 0.056481752544641495
[MASK]: 'عموما'  : 0.0195249505341053
متناظر متناظرا قبل قبلا
  متناظر صفت و متناظرا قید است. در نتیجه قبل صفت و [MASK] قید است.
[MASK]: 'نیز'  : 0.06125229597091675
[MASK]: 'هم'  : 0.06033962965011597
[MASK]: 'واقع'  : 0.0206016

In [37]:
Accuracy = 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتفاق اتفاقا شب شبانه
 نسبت اتفاق به اتفاقا مانند نسبت شب به [MASK] است.
[MASK]: 'هم'  : 0.04602111503481865
[MASK]: 'منزل'  : 0.03534402325749397
[MASK]: 'من'  : 0.02838587388396263
حتم حتما اساس اساسا
 نسبت حتم به حتما مانند نسبت اساس به [MASK] است.
[MASK]: 'این'  : 0.05438289791345596
[MASK]: 'من'  : 0.020888889208436012
[MASK]: 'درستی'  : 0.012288955971598625
اصل اصلا انسان انسانوار
 نسبت اصل به اصلا مانند نسبت انسان به [MASK] است.
[MASK]: 'هم'  : 0.03388819843530655
[MASK]: 'اصلا'  : 0.02929743565618992
[MASK]: 'خوبی'  : 0.026171742007136345
عمل عملا اجبار اجبارا
 نسبت عمل به عملا مانند نسبت اجبار به [MASK] است.
[MASK]: 'عمل'  : 0.15050671994686127
[MASK]: 'کار'  : 0.047876711934804916
[MASK]: 'تنهایی'  : 0.0325179323554039
اجبار اجبارا کودک کودکانه
 نسبت اجبار به اجبارا مانند نسبت کودک به [MASK] است.
[MASK]: 'مادر'  : 0.0640394315123558
[MASK]: 'خارج'  : 0.05475707724690437
[MASK]: 'عقب'  : 0.036900926381349564
دائم دائما اتفاق اتفاقا
 نسبت دائم به دائما مانند نسبت اتفاق به [MASK

In [38]:
Accuracy = 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

خشنود ناخشنود موفق ناموفق
خشنود به ناخشنود مانند موفق به [MASK] است.
[MASK]: 'فرار'  : 0.057805925607681274
[MASK]: 'خوبی'  : 0.031425703316926956
[MASK]: 'کار'  : 0.029986172914505005
درست نادرست پخته ناپخته
درست به نادرست مانند پخته به [MASK] است.
[MASK]: 'غلط'  : 0.04622553288936615
[MASK]: 'اشتباه'  : 0.044795095920562744
[MASK]: 'خوبی'  : 0.031099582090973854
مناسب نامناسب همزمان ناهمزمان
مناسب به نامناسب مانند همزمان به [MASK] است.
[MASK]: 'هم'  : 0.09575830399990082
[MASK]: 'حرکت'  : 0.0410570427775383
[MASK]: 'کار'  : 0.030763985589146614
مربوط نامربوط منسجم نامنسجم
مربوط به نامربوط مانند منسجم به [MASK] است.
[MASK]: 'خود'  : 0.01832140050828457
[MASK]: 'کردن'  : 0.015504471026360989
[MASK]: 'کار'  : 0.014971167780458927
بینا نابینا درست نادرست
بینا به نابینا مانند درست به [MASK] است.
[MASK]: 'نفس'  : 0.08050503581762314
[MASK]: 'دراوردن'  : 0.06790715456008911
[MASK]: 'کار'  : 0.028358520939946175
کارآمد ناکارآمد هنجار ناهنجار
کارآمد به ناکارآمد مانند هنجار به [MASK] است.
[MAS

In [39]:
Accuracy = 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

سریع سریعتر بلند بلندتر
 نسبت سریع به سریعتر مانند نسبت بلند به [MASK] است.
[MASK]: 'کوتاه'  : 0.11442290991544724
[MASK]: 'بلند'  : 0.06601317971944809
[MASK]: 'اهسته'  : 0.045127224177122116
بزرگ بزرگتر شدید شدیدتر
 نسبت بزرگ به بزرگتر مانند نسبت شدید به [MASK] است.
[MASK]: 'شدید'  : 0.01811690255999565
[MASK]: 'راحتی'  : 0.013096977956593037
[MASK]: 'بدن'  : 0.011055177077651024
مهم مهمتر بارز بارزتر
 نسبت مهم به مهمتر مانند نسبت بارز به [MASK] است.
[MASK]: 'مهمتر'  : 0.052200302481651306
[MASK]: 'خوبی'  : 0.040613338351249695
[MASK]: 'مسلم'  : 0.02457309700548649
به بهتر سرد سردتر
 نسبت به به بهتر مانند نسبت سرد به [MASK] است.
[MASK]: 'بهتر'  : 0.0796714723110199
[MASK]: 'خوبی'  : 0.07506208121776581
[MASK]: 'گرم'  : 0.06966165453195572
واضح واضحتر بیش بیشتر
 نسبت واضح به واضحتر مانند نسبت بیش به [MASK] است.
[MASK]: 'حد'  : 0.26966017484664917
[MASK]: 'اندازه'  : 0.08223932236433029
[MASK]: 'حدی'  : 0.04514759033918381
کم کمتر بد بدتر
 نسبت کم به کمتر مانند نسبت بد به [MASK] است.
[

In [40]:
Accuracy = 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " مردم " + df["second"][i[0]] + " اهل کشور " + df["first"][i[0]] + " هستند. و مردم " + "[MASK]" + " اهل کشور " + df["third"][i[0]] + " هستند."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتریش اتریشی اسپانیا اسپانیایی
 مردم اتریشی اهل کشور اتریش هستند. و مردم [MASK] اهل کشور اسپانیا هستند.
[MASK]: 'اتریش'  : 0.2926158905029297
[MASK]: 'اتریشی'  : 0.1856233775615692
[MASK]: 'ان'  : 0.03602100908756256
سوئد سوئدی فنلاند فنلاندی
 مردم سوئدی اهل کشور سوئد هستند. و مردم [MASK] اهل کشور فنلاند هستند.
[MASK]: 'سويدی'  : 0.8454741835594177
[MASK]: 'سويد'  : 0.033914048224687576
[MASK]: 'فنلاندی'  : 0.00828893855214119
بحرین بحرینی عراق عراقی
 مردم بحرینی اهل کشور بحرین هستند. و مردم [MASK] اهل کشور عراق هستند.
[MASK]: 'بحرینی'  : 0.9213269352912903
[MASK]: 'بحرین'  : 0.029526326805353165
[MASK]: 'بوکانی'  : 0.0017833971651270986
کویت کویتی سوریه سوری
 مردم کویتی اهل کشور کویت هستند. و مردم [MASK] اهل کشور سوریه هستند.
[MASK]: 'کویتی'  : 0.45752692222595215
[MASK]: 'سوریه'  : 0.0379827618598938
[MASK]: 'ان'  : 0.03453085571527481
مصر مصری کرواسی کروات
 مردم مصری اهل کشور مصر هستند. و مردم [MASK] اهل کشور کرواسی هستند.
[MASK]: 'مصری'  : 0.19172844290733337
[MASK]: 'ان'  : 0.0533

In [41]:
Accuracy = 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " " + df["first"][i[0]] + " جمع " + df["second"][i[0]] + " است و " + df["third"][i[0]] + " جمع " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

تالیف تالیفات مخلوق مخلوقات
 تالیف جمع تالیفات است و مخلوق جمع [MASK] است.
[MASK]: 'نیز'  : 0.1356441080570221
[MASK]: 'هم'  : 0.09721354395151138
[MASK]: 'شده'  : 0.093879334628582
اثر اثرات ارزش ارزشها
 اثر جمع اثرات است و ارزش جمع [MASK] است.
[MASK]: 'شده'  : 0.06082827225327492
[MASK]: 'ارزش'  : 0.05544663220643997
[MASK]: 'نیز'  : 0.03648487105965614
انقلابی انقلابیون مورد موارد
 انقلابی جمع انقلابیون است و مورد جمع [MASK] است.
[MASK]: 'ایشان'  : 0.10478024184703827
[MASK]: 'انقلاب'  : 0.06709859520196915
[MASK]: 'ما'  : 0.03453058749437332
حواری حواریون مورد موارد
 حواری جمع حواریون است و مورد جمع [MASK] است.
[MASK]: 'ایشان'  : 0.11600790917873383
[MASK]: 'نیز'  : 0.03063337504863739
[MASK]: 'انان'  : 0.015368836000561714
مسئول مسئولین آزاده آزادگان
 مسئول جمع مسئولین است و آزاده جمع [MASK] است.
[MASK]: 'شده'  : 0.15238194167613983
[MASK]: 'ایشان'  : 0.0414021760225296
[MASK]: 'ما'  : 0.036577824503183365
مسافر مسافرین طلا طلاها
 مسافر جمع مسافرین است و طلا جمع [MASK] است.
[MASK]

**تست روش دوم**

In [43]:
Accuracy= 0
y= df[df["cat"]=="city"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " "+df["first"][i[0]] + " در " + "[MASK]" + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = " "+df["third"][i[0]] + " در " + predicted + "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)      


اهواز خوزستان قشم هرمزگان
 اهواز در [MASK]خوزستان است.
[MASK]: 'استان'  : 0.3800472021102905
[MASK]: 'کشور'  : 0.13085097074508667
[MASK]: 'واقع'  : 0.11511075496673584
 قشم در استان[MASK] است.
[MASK]: 'هرمزگان'  : 0.3603532910346985
[MASK]: 'اذربایجان'  : 0.054229915142059326
[MASK]: 'خوزستان'  : 0.03675452247262001
شیراز فارس دامغان سمنان
 شیراز در [MASK]فارس است.
[MASK]: 'کشور'  : 0.12228133529424667
[MASK]: 'واقع'  : 0.04899128898978233
[MASK]: 'استان'  : 0.0472029447555542
 دامغان در کشور[MASK] است.
[MASK]: 'ایران'  : 0.36921241879463196
[MASK]: 'اذربایجان'  : 0.06045302748680115
[MASK]: 'جهت'  : 0.051247186958789825
رشت گیلان اسلام‌شهر تهران
 رشت در [MASK]گیلان است.
[MASK]: 'استان'  : 0.2839483320713043
[MASK]: 'کشور'  : 0.13477995991706848
[MASK]: 'واقع'  : 0.053787730634212494
 اسلام‌شهر در استان[MASK] است.
[MASK]: 'کرمان'  : 0.11281871795654297
[MASK]: 'خوزستان'  : 0.09616933017969131
[MASK]: 'هرمزگان'  : 0.07550037652254105
بندرعباس هرمزگان ملارد تهران
 بندرعباس در [MASK]هرمز

In [44]:
Accuracy= 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = " "+"[MASK]" + " " + predicted + " " + df["third"][i[0]] + " است."
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

روسیه مسکو هلند آمستردام
مسکو [MASK] روسیه است.
[MASK]: '،'  : 0.03982507064938545
[MASK]: 'سفارت'  : 0.034875474870204926
[MASK]: 'کشور'  : 0.025448137894272804
 [MASK] ، هلند است.
[MASK]: 'دیگر'  : 0.017315339297056198
[MASK]: 'منظور'  : 0.01539486926048994
[MASK]: 'هلند'  : 0.014181259088218212
کره‌جنوبی سئول آمریکا واشنگتن
سئول [MASK] کره‌جنوبی است.
[MASK]: 'پایتخت'  : 0.2776593565940857
[MASK]: 'کشور'  : 0.2231006771326065
[MASK]: '،'  : 0.09931362420320511
 [MASK] پایتخت آمریکا است.
[MASK]: 'امروز'  : 0.04563681036233902
[MASK]: 'اکنون'  : 0.020329466089606285
[MASK]: 'امروزه'  : 0.01737542636692524
ایران تهران اسپانیا مادرید
تهران [MASK] ایران است.
[MASK]: 'پایتخت'  : 0.18239378929138184
[MASK]: 'امروز'  : 0.04150853678584099
[MASK]: 'تهران'  : 0.03605877235531807
 [MASK] پایتخت اسپانیا است.
[MASK]: 'مادرید'  : 0.015470171347260475
[MASK]: 'امروز'  : 0.015161311253905296
[MASK]: 'حکم'  : 0.013339911587536335
ایران تهران افغانستان کابل
تهران [MASK] ایران است.
[MASK]: 'پایتخت'  : 

In [45]:
Accuracy= 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

سوئد کرون امارات درهم
کرون [MASK] سوئد است.
[MASK]: 'کشور'  : 0.8843193054199219
[MASK]: 'در'  : 0.009176437743008137
[MASK]: 'ملی'  : 0.006562130060046911
کشورامارات [MASK] است.
[MASK]: 'کشور'  : 0.041767820715904236
[MASK]: 'ایران'  : 0.0254669226706028
[MASK]: 'مستقل'  : 0.0236701350659132
سوئیس فرانک تاجیکستان سامانی
فرانک [MASK] سوئیس است.
[MASK]: 'کشور'  : 0.40448737144470215
[MASK]: '،'  : 0.07315290719270706
[MASK]: 'از'  : 0.04229586198925972
کشورتاجیکستان [MASK] است.
[MASK]: 'چگونه'  : 0.0755627378821373
[MASK]: 'ازاد'  : 0.025412460789084435
[MASK]: 'مشخص'  : 0.02038516476750374
رومانی لئو بریتانیا پوند
لئو [MASK] رومانی است.
[MASK]: 'اول'  : 0.0473988801240921
[MASK]: 'در'  : 0.03296244144439697
[MASK]: 'امروز'  : 0.031612515449523926
اولبریتانیا [MASK] است.
[MASK]: 'واقعی'  : 0.021244466304779053
[MASK]: 'کامل'  : 0.0204035472124815
[MASK]: 'مشخص'  : 0.019356315955519676
ترکیه لیره افغانستان افغانی
لیره [MASK] ترکیه است.
[MASK]: 'ایران'  : 0.10673586279153824
[MASK]: 'رسمی

In [46]:
Accuracy= 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برادر خواهر پدر مادر
خواهر [MASK] برادر است.
[MASK]: 'هم'  : 0.07700567692518234
[MASK]: 'نیز'  : 0.07155994325876236
[MASK]: 'دیگر'  : 0.07047899812459946
همپدر [MASK] است. 
[MASK]: 'کار'  : 0.04046649485826492
[MASK]: 'منزل'  : 0.03879852965474129
[MASK]: 'خانه'  : 0.02476564235985279
برادرها خواهرها شوهر عیال
خواهرها [MASK] برادرها است.
[MASK]: 'و'  : 0.04596610739827156
[MASK]: 'از'  : 0.01888297125697136
[MASK]: 'دست'  : 0.01745682768523693
وشوهر [MASK] است. 
[MASK]: 'خوب'  : 0.014209485612809658
[MASK]: 'خوبی'  : 0.012724556028842926
[MASK]: 'مردان'  : 0.011243239045143127
پدربزرگ مادربزرگ برادر خواهر
مادربزرگ [MASK] پدربزرگ است.
[MASK]: '،'  : 0.23624587059020996
[MASK]: 'نیز'  : 0.1153009831905365
[MASK]: 'هم'  : 0.05127035081386566
،برادر [MASK] است. 
[MASK]: 'برادرش'  : 0.05281848460435867
[MASK]: 'پدر'  : 0.039200376719236374
[MASK]: 'من'  : 0.03728385269641876
بابابزرگ مامان‌بزرگ داماد عروس
مامان‌بزرگ [MASK] بابابزرگ است.
[MASK]: '،'  : 0.28143933415412903
[MASK]: 'مثل'  : 

In [47]:
Accuracy= 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["second"][i[0]] + " به " +df["first"][i[0]]  + " " + "[MASK]"+ " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برید بریدند رقصید رقصیدند
 نسبت بریدند به برید [MASK] است.
[MASK]: 'کم'  : 0.056336909532547
[MASK]: 'مشخص'  : 0.04492636397480965
[MASK]: 'یکسان'  : 0.03656161203980446
کمرقصید [MASK] است. 
[MASK]: 'اشتباه'  : 0.037982095032930374
[MASK]: 'مشخص'  : 0.034350160509347916
[MASK]: 'طبیعی'  : 0.02047087624669075
پرید پریدند آمد آمدند
 نسبت پریدند به پرید [MASK] است.
[MASK]: 'بیشتر'  : 0.035055581480264664
[MASK]: 'صفر'  : 0.03341984748840332
[MASK]: 'یکسان'  : 0.030039124190807343
بیشترآمد [MASK] است. 
[MASK]: 'مردان'  : 0.14055441319942474
[MASK]: 'طبیعی'  : 0.04641103744506836
[MASK]: 'زنان'  : 0.03869301825761795
تازید تازیدند گفت گفتند
 نسبت تازیدند به تازید [MASK] است.
[MASK]: 'بیشتر'  : 0.04653143510222435
[MASK]: 'یکسان'  : 0.04261854663491249
[MASK]: 'متفاوت'  : 0.038417696952819824
بیشترگفت [MASK] است. 
[MASK]: 'من'  : 0.02981751225888729
[MASK]: 'خانواده'  : 0.0169682577252388
[MASK]: 'دولت'  : 0.01610403321683407
شد شدند بوسید بوسیدند
 نسبت شدند به شد [MASK] است.
[MASK]: 'بوده' 

In [48]:
Accuracy= 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

بریدن برید دمیدن دمید
برید [MASK] بریدن است.
[MASK]: '،'  : 0.44085896015167236
[MASK]: 'همان'  : 0.09210815280675888
[MASK]: 'و'  : 0.028073938563466072
،دمیدن [MASK] است. 
[MASK]: 'عمدی'  : 0.012097304686903954
[MASK]: 'ازاد'  : 0.00956727471202612
[MASK]: 'طبیعی'  : 0.00929216481745243
تازیدن تاخت دادن داد
تاخت [MASK] تازیدن است.
[MASK]: '،'  : 0.2761690616607666
[MASK]: 'بهمعنای'  : 0.08324576914310455
[MASK]: 'نوعی'  : 0.08071477711200714
،دادن [MASK] است. 
[MASK]: 'خوب'  : 0.04121379181742668
[MASK]: 'اختیاری'  : 0.04053343087434769
[MASK]: 'وظیفه'  : 0.03467648848891258
شدن شد نگریستن نگریست
شد [MASK] شدن است.
[MASK]: 'تمام'  : 0.016446715220808983
[MASK]: 'بزرگ'  : 0.01503939088433981
[MASK]: 'خیلی'  : 0.01463923417031765
تمامنگریستن [MASK] است. 
[MASK]: 'اشتباه'  : 0.03538147360086441
[MASK]: 'گناه'  : 0.019873041659593582
[MASK]: 'همین'  : 0.019342219457030296
گفتن گفت رقصیدن رقصید
گفت [MASK] گفتن است.
[MASK]: '،'  : 0.1515713930130005
[MASK]: 'حرف'  : 0.034317344427108765
[M

In [49]:
Accuracy= 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

دادم دادیم یافتم یافتیم
دادیم [MASK] دادم است.
[MASK]: '،'  : 0.174472838640213
[MASK]: 'و'  : 0.13005144894123077
[MASK]: 'هم'  : 0.05723641440272331
،یافتم [MASK] است. 
[MASK]: 'من'  : 0.06740038096904755
[MASK]: 'هم'  : 0.017686963081359863
[MASK]: '،'  : 0.016323719173669815
زدم زدیم نامیدم نامیدیم
زدیم [MASK] زدم است.
[MASK]: 'و'  : 0.253645122051239
[MASK]: '،'  : 0.10852266848087311
[MASK]: 'هم'  : 0.08982185274362564
ونامیدم [MASK] است. 
[MASK]: 'هم'  : 0.07677453756332397
[MASK]: 'نیز'  : 0.06327566504478455
[MASK]: 'مشخص'  : 0.01916552521288395
نوشتم نوشتیم گفتم گفتیم
نوشتیم [MASK] نوشتم است.
[MASK]: '،'  : 0.17092880606651306
[MASK]: 'را'  : 0.15593434870243073
[MASK]: 'هم'  : 0.09450416266918182
،گفتم [MASK] است. 
[MASK]: 'درست'  : 0.0674092099070549
[MASK]: 'خوب'  : 0.05695265159010887
[MASK]: 'اشتباه'  : 0.024978790432214737
دانستم دانستیم بلعیدم بلعیدیم
دانستیم [MASK] دانستم است.
[MASK]: '،'  : 0.14318807423114777
[MASK]: '؟'  : 0.0879959762096405
[MASK]: 'که'  : 0.06607

In [50]:
Accuracy= 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

مجدد مجددا مخصوص مخصوصا
مجددا [MASK] مجدد است.
[MASK]: 'سازی'  : 0.0822327584028244
[MASK]: '،'  : 0.05977290868759155
[MASK]: 'این'  : 0.03054460883140564
سازیمخصوص [MASK] است. 
[MASK]: 'انسان'  : 0.015245640650391579
[MASK]: 'خانواده'  : 0.014900299720466137
[MASK]: 'ازاد'  : 0.007582970429211855
عمده عمدتا صمیمی صمیمانه
عمدتا [MASK] عمده است.
[MASK]: 'کار'  : 0.026167238131165504
[MASK]: 'نقش'  : 0.025408798828721046
[MASK]: 'بار'  : 0.012272946536540985
کارصمیمی [MASK] است. 
[MASK]: 'خوب'  : 0.12691931426525116
[MASK]: 'اشتباه'  : 0.07126735895872116
[MASK]: 'هم'  : 0.03499782085418701
دقیق دقیقا عادل عادلانه
دقیقا [MASK] دقیق است.
[MASK]: 'هم'  : 0.197697713971138
[MASK]: 'خیلی'  : 0.17514429986476898
[MASK]: 'کاملا'  : 0.03267693147063255
همعادل [MASK] است. 
[MASK]: 'برعکس'  : 0.03540844842791557
[MASK]: 'خوب'  : 0.030334940180182457
[MASK]: 'مطلق'  : 0.026557451114058495
غالب غالبا جدا جداگانه
غالبا [MASK] غالب است.
[MASK]: 'رنگ'  : 0.022907357662916183
[MASK]: 'صوت'  : 0.019126

In [51]:
Accuracy= 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

اتفاق اتفاقا شب شبانه
اتفاقا [MASK] اتفاق است.
[MASK]: 'این'  : 0.30739882588386536
[MASK]: 'هم'  : 0.23795931041240692
[MASK]: 'همین'  : 0.05839301645755768
اینشب [MASK] است. 
[MASK]: 'عمومی'  : 0.015337199904024601
[MASK]: 'سنگین'  : 0.013318285346031189
[MASK]: 'تعطیل'  : 0.011842932552099228
حتم حتما اساس اساسا
حتما [MASK] حتم است.
[MASK]: 'هم'  : 0.6230632662773132
[MASK]: '-'  : 0.029294617474079132
[MASK]: '،'  : 0.02439149282872677
هماساس [MASK] است. 
[MASK]: 'ایرانی'  : 0.0405474454164505
[MASK]: 'کفیل'  : 0.03838334232568741
[MASK]: 'وکیل'  : 0.026637859642505646
اصل اصلا انسان انسانوار
اصلا [MASK] اصل است.
[MASK]: 'این'  : 0.10039987415075302
[MASK]: 'اینها'  : 0.03818516433238983
[MASK]: 'هم'  : 0.02508382871747017
اینانسان [MASK] است. 
[MASK]: 'نیز'  : 0.011347318068146706
[MASK]: 'سبک'  : 0.01035627443343401
[MASK]: 'خوب'  : 0.010043518617749214
عمل عملا اجبار اجبارا
عملا [MASK] عمل است.
[MASK]: 'ملاک'  : 0.04318997263908386
[MASK]: 'انسان'  : 0.04180362820625305
[MASK]: 

In [52]:
Accuracy= 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " و " + df["second"][i[0]]+"[MASK]" + " یکدیگر هستند."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

خشنود ناخشنود موفق ناموفق
خشنود و ناخشنود[MASK] یکدیگر هستند.
[MASK]: 'دشمنان'  : 0.22524791955947876
[MASK]: 'دشمن'  : 0.04924147576093674
[MASK]: 'پشتیبان'  : 0.03201906755566597
دشمنانموفق [MASK] است. 
[MASK]: 'شده'  : 0.17985641956329346
[MASK]: 'کامل'  : 0.029114825651049614
[MASK]: 'دشمن'  : 0.027353039011359215
درست نادرست پخته ناپخته
درست و نادرست[MASK] یکدیگر هستند.
[MASK]: 'علیه'  : 0.11686944216489792
[MASK]: 'از'  : 0.05302637815475464
[MASK]: 'دشمنان'  : 0.02506539225578308
علیهپخته [MASK] است. 
[MASK]: 'شده'  : 0.030605439096689224
[MASK]: 'موثر'  : 0.01995498687028885
[MASK]: 'خوب'  : 0.016885964199900627
مناسب نامناسب همزمان ناهمزمان
مناسب و نامناسب[MASK] یکدیگر هستند.
[MASK]: 'دشمنان'  : 0.11124681681394577
[MASK]: 'شان'  : 0.05193450674414635
[MASK]: 'از'  : 0.043217405676841736
دشمنانهمزمان [MASK] است. 
[MASK]: 'خوب'  : 0.04629507660865784
[MASK]: 'خنثی'  : 0.03842267021536827
[MASK]: 'ما'  : 0.02525508590042591
مربوط نامربوط منسجم نامنسجم
مربوط و نامربوط[MASK] یکدیگ

In [53]:
Accuracy= 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ "[MASK]" + " "   +  df["third"][i[0]] + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

سریع سریعتر بلند بلندتر
سریعتر [MASK] سریع است.
[MASK]: 'و'  : 0.2606126368045807
[MASK]: '،'  : 0.2435753345489502
[MASK]: 'یا'  : 0.10100886225700378
و[MASK] بلند است. 
[MASK]: 'صدایش'  : 0.024194663390517235
[MASK]: 'همواره'  : 0.022902797907590866
[MASK]: 'بسیار'  : 0.018303586170077324
بزرگ بزرگتر شدید شدیدتر
بزرگتر [MASK] بزرگ است.
[MASK]: 'از'  : 0.6056355834007263
[MASK]: '،'  : 0.11059904843568802
[MASK]: 'و'  : 0.082618847489357
از[MASK] شدید است. 
[MASK]: 'دید'  : 0.060616374015808105
[MASK]: 'کارافتادگی'  : 0.059029776602983475
[MASK]: 'نوع'  : 0.02976393885910511
مهم مهمتر بارز بارزتر
مهمتر [MASK] مهم است.
[MASK]: 'از'  : 0.10970862954854965
[MASK]: 'هم'  : 0.10028444230556488
[MASK]: 'اینکه'  : 0.09453805536031723
از[MASK] بارز است. 
[MASK]: 'نوع'  : 0.042375534772872925
[MASK]: 'جمله'  : 0.021818464621901512
[MASK]: 'اشکال'  : 0.018017109483480453
به بهتر سرد سردتر
بهتر [MASK] به است.
[MASK]: 'از'  : 0.7077842950820923
[MASK]: 'است'  : 0.081451915204525
[MASK]: 'بودن'  :

In [54]:
Accuracy= 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+" "+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

اتریش اتریشی اسپانیا اسپانیایی
اتریشی [MASK] اتریش است.
[MASK]: '،'  : 0.3003113269805908
[MASK]: 'را'  : 0.031541429460048676
[MASK]: 'نام'  : 0.024013109505176544
، اسپانیا [MASK] است. 
[MASK]: 'من'  : 0.028220228850841522
[MASK]: 'ما'  : 0.02437432110309601
[MASK]: 'ایران'  : 0.02078123204410076
سوئد سوئدی فنلاند فنلاندی
سوئدی [MASK] سوئد است.
[MASK]: '،'  : 0.04572460800409317
[MASK]: 'کشور'  : 0.03629614785313606
[MASK]: 'را'  : 0.03179645538330078
، فنلاند [MASK] است. 
[MASK]: 'من'  : 0.022466478869318962
[MASK]: 'ما'  : 0.022152382880449295
[MASK]: 'ایران'  : 0.015322341583669186
بحرین بحرینی عراق عراقی
بحرینی [MASK] بحرین است.
[MASK]: 'کشور'  : 0.14718016982078552
[MASK]: '،'  : 0.04529900848865509
[MASK]: 'از'  : 0.02658757008612156
کشور عراق [MASK] است. 
[MASK]: 'ساقط'  : 0.03732428699731827
[MASK]: 'غیرقانونی'  : 0.025952132418751717
[MASK]: 'کجا'  : 0.022488676011562347
کویت کویتی سوریه سوری
کویتی [MASK] کویت است.
[MASK]: '،'  : 0.06422411650419235
[MASK]: 'امروز'  : 0.0400

In [55]:
Accuracy= 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+ df["third"][i[0]] + " " + predicted  +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

تالیف تالیفات مخلوق مخلوقات
تالیفات [MASK] تالیف است.
[MASK]: 'اولین'  : 0.043620578944683075
[MASK]: 'من'  : 0.037302352488040924
[MASK]: 'مهمترین'  : 0.02013486623764038
مخلوق اولین[MASK] است. 
[MASK]: 'انسان'  : 0.17410413920879364
[MASK]: 'بار'  : 0.07645415514707565
[MASK]: 'ما'  : 0.03467053547501564
اثر اثرات ارزش ارزشها
اثرات [MASK] اثر است.
[MASK]: 'تشخیص'  : 0.02175520732998848
[MASK]: 'روی'  : 0.010744025930762291
[MASK]: 'این'  : 0.008008589036762714
ارزش تشخیص[MASK] است. 
[MASK]: 'شده'  : 0.13745087385177612
[MASK]: 'نشده'  : 0.049208056181669235
[MASK]: 'ما'  : 0.021023806184530258
انقلابی انقلابیون مورد موارد
انقلابیون [MASK] انقلابی است.
[MASK]: '،'  : 0.06762588024139404
[MASK]: 'ما'  : 0.05456772819161415
[MASK]: 'ایران'  : 0.04263013228774071
مورد ،[MASK] است. 
[MASK]: 'من'  : 0.0266560185700655
[MASK]: 'خوب'  : 0.014009448699653149
[MASK]: 'شما'  : 0.0076108528301119804
حواری حواریون مورد موارد
حواریون [MASK] حواری است.
[MASK]: 'نیز'  : 0.27520790696144104
[MASK]: '